# Notebook 05: Dashboard Integration & Deployment Readiness

**Author:** Hector Carbajal  
**Version:** 1.2 (Production Patch)  
**Last Updated:** 2026-02

---

## Purpose
This notebook serves as the **final gateway** before production deployment. It performs data pre-aggregation for the Streamlit dashboard and executes a health check on all exported assets to ensure sub-second loading times for stakeholders.

## Inputs
- `data/processed/trips_with_efficiency.csv` - Scored trip data

## Outputs
- `data/processed/dashboard/regional_summary.csv` - Aggregated region metrics
- `data/processed/dashboard/hourly_efficiency.csv` - Time-series trend data

## Deployment Readiness Checks
- **Latency Check**: Are data files small enough for fast loading (< 5MB)?
- **Memory Deep Audit**: Deep inspection of DataFrame memory usage.
- **Aggregation Validation**: Confirming metrics preserve source-of-truth integrity.

---

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os

# Project imports
sys.path.insert(0, str(Path.cwd().parent))
from src.config import PROCESSED_DIR

DASHBOARD_DATA_DIR = PROCESSED_DIR / 'dashboard'
DASHBOARD_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Setup complete. Pre-aggregation target: {DASHBOARD_DATA_DIR}")

✅ Setup complete. Pre-aggregation target: data/processed/dashboard/


## 1. Data Pre-Aggregation
*Transforming raw ledgers into optimized dashboard assets.*

In [2]:
trips_df = pd.read_csv(PROCESSED_DIR / 'trips_with_efficiency.csv')

# Regional Summary
regional_summary = trips_df.groupby('region').agg({
    'trip_id': 'count',
    'efficiency_index': 'mean',
    'is_cancelled': 'sum',
    'is_late_pickup': 'sum'
}).rename(columns={'trip_id': 'total_trips'})
regional_summary['cancellation_rate'] = regional_summary['is_cancelled'] / regional_summary['total_trips']
regional_summary.to_csv(DASHBOARD_DATA_DIR / 'regional_summary.csv')

# Hourly Efficiency Trend
hourly_efficiency = trips_df.groupby('scheduled_hour')['efficiency_index'].mean().reset_index()
hourly_efficiency.to_csv(DASHBOARD_DATA_DIR / 'hourly_efficiency.csv', index=False)

print("✅ Regional summary exported.")
print("✅ Hourly efficiency trend exported.")

✅ Regional summary exported.
✅ Hourly efficiency trend exported.


## 2. Dashboard Health & Integrity Audit
*Deep inspection of export assets.*

In [ ]:
print("🔍 ASSET AUDIT:")

# Store audit results for insights computation
audit_results = {}

def verify_export(df, name, is_aggregated=True):
    """Deep memory inspection with comprehensive metrics."""
    mem_bytes = df.memory_usage(deep=True).sum()
    size_mb = mem_bytes / (1024 * 1024)
    row_count = len(df)
    col_count = len(df.columns)
    status = "⚡ FAST" if size_mb < 5 else "⚠️ SLOW"
    print(f"- {name:20} | {row_count:>6,} rows | {col_count:>2} cols | {size_mb:>6.3f} MB | {status}")
    audit_results[name] = {'size_mb': size_mb, 'rows': row_count, 'cols': col_count}
    return size_mb

# Audit aggregated files
regional_size = verify_export(regional_summary, "Regional Summary")
hourly_size = verify_export(hourly_efficiency, "Hourly Efficiency")

# Calculate actual source file size for comparison
source_size_mb = trips_df.memory_usage(deep=True).sum() / (1024 * 1024)
total_aggregated_size = regional_size + hourly_size
reduction_pct = ((source_size_mb - total_aggregated_size) / source_size_mb) * 100 if source_size_mb > 0 else 0

print(f"\n📊 AGGREGATION IMPACT:")
print(f"- Source data size:     {source_size_mb:.3f} MB ({len(trips_df):,} rows)")
print(f"- Aggregated data size: {total_aggregated_size:.3f} MB")
print(f"- Memory reduction:     {reduction_pct:.1f}%")

# Store for insights cell
deployment_metrics = {
    'source_size_mb': source_size_mb,
    'aggregated_size_mb': total_aggregated_size,
    'reduction_pct': reduction_pct,
    'source_rows': len(trips_df),
    'all_fast': regional_size < 5 and hourly_size < 5
}

## 💡 Key Findings
**Deployment Strategy for NEMT Dashboard**

In [ ]:
# Use actual computed metrics for insights
print("="*80)
print("🚀 DASHBOARD DEPLOYMENT PLAN")
print("="*80)

print(f"\n1. PERFORMANCE:")
print(f"   Pre-aggregation reduces dashboard memory by {deployment_metrics['reduction_pct']:.0f}%")
print(f"   ({deployment_metrics['source_size_mb']:.2f} MB → {deployment_metrics['aggregated_size_mb']:.3f} MB)")
print(f"   All assets pass latency check: {'✅ YES' if deployment_metrics['all_fast'] else '❌ NO'}")

print(f"\n2. DATA REFRESH:")
print(f"   Source data: {deployment_metrics['source_rows']:,} trip records")
print(f"   Recommendation: Schedule this notebook daily at 02:00 AM UTC for fresh aggregations")

print(f"\n3. STAKEHOLDER ACCESS:")
print(f"   Dashboard assets exported to: {DASHBOARD_DATA_DIR}")
print(f"   The 'Driver Coaching' view is ready for HR consumption")
print(f"   Credentials: Managed via Streamlit Secrets (st.secrets)")

print("\n" + "="*80)